In [1]:
from tensorflow import keras
from functools import partial

StandardLayer = partial(keras.layers.Dense, activation="elu", kernel_initializer="he_normal")

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

In [3]:
X_train_full.shape

(50000, 32, 32, 3)

In [4]:
X_train_full[0].max()

255

In [5]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [6]:
y_train.shape

(45000, 1)

In [7]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))

for i in range(0, 20):
    model.add(StandardLayer(100))

model.add(keras.layers.Dense(10, activation="softmax", kernel_initializer="he_normal"))

In [8]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.Nadam(lr=5e-5),
             metrics=["accuracy"])

In [9]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d_-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [11]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6007

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

KeyboardInterrupt: 

In [12]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid,y_valid), callbacks=callbacks)

Epoch 1/100
   1/1407 [..............................] - ETA: 1s - loss: 116.2598 - accuracy: 0.1250WARNING:tensorflow:From C:\Users\deaston\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1407/1407 [==============================] - 12s 8ms/step - loss: 4.0447 - accuracy: 0.1708 - val_loss: 2.6408 - val_accuracy: 0.1680
Epoch 2/100
1407/1407 [==============================] - 12s 8ms/step - loss: 2.0847 - accuracy: 0.2398 - val_loss: 2.0217 - val_accuracy: 0.2656
Epoch 3/100
1407/1407 [==============================] - 12s 8ms/step - loss: 1.9703 - accuracy: 0.2840 - val_loss: 1.9312 - val_accuracy: 0.3084
Epoch 4/100
1407/1407 [==============================] - 12s 8ms/step - loss: 1.8907 - accuracy: 0.3131 - val_loss: 1.9108 - val_accuracy: 0.2924
Epoch 5/100
1407/1407 [=====================

1407/1407 [==============================] - 12s 8ms/step - loss: 1.1713 - accuracy: 0.5792 - val_loss: 1.5559 - val_accuracy: 0.4702
Epoch 53/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.1657 - accuracy: 0.5826 - val_loss: 1.5893 - val_accuracy: 0.4692
Epoch 54/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.1623 - accuracy: 0.5842 - val_loss: 1.5575 - val_accuracy: 0.4738
Epoch 55/100
1407/1407 [==============================] - 11s 8ms/step - loss: 1.1517 - accuracy: 0.5896 - val_loss: 1.5762 - val_accuracy: 0.4718


In [13]:
bn_model = keras.models.Sequential()
bn_model.add(keras.layers.Flatten(input_shape=[32,32,3]))

for i in range(0, 20):
    bn_model.add(keras.layers.BatchNormalization())
    bn_model.add(StandardLayer(100))

bn_model.add(keras.layers.Dense(10, activation="softmax", kernel_initializer="he_normal"))

In [16]:
bn_model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.Nadam(lr=5e-4),
             metrics=["accuracy"])

In [17]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("bn_cifar10_model.h5", save_best_only=True)
run_index = 2 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "bn_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]
history = bn_model.fit(X_train, y_train, epochs=100, validation_data=(X_valid,y_valid), callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 26s 18ms/step - loss: 1.5787 - accuracy: 0.4444 - val_loss: 1.5537 - val_accuracy: 0.4442
Epoch 2/100
1407/1407 [==============================] - 19s 13ms/step - loss: 1.5163 - accuracy: 0.4661 - val_loss: 1.5053 - val_accuracy: 0.4718
Epoch 3/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.4838 - accuracy: 0.4727 - val_loss: 1.4692 - val_accuracy: 0.4824
Epoch 4/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.4458 - accuracy: 0.4906 - val_loss: 1.4452 - val_accuracy: 0.4900
Epoch 5/100
1407/1407 [==============================] - 19s 13ms/step - loss: 1.4106 - accuracy: 0.5026 - val_loss: 1.4311 - val_accuracy: 0.4950
Epoch 6/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.3831 - accuracy: 0.5132 - val_loss: 1.4456 - val_accuracy: 0.4868
Epoch 7/100
1407/1407 [==============================] - 19s 13ms/step - loss: 1.3637 - accuracy: 0.5199 - val_loss: 1

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]